# Programowanie obiektowe
> I thought of objects being like biological cells and/or individual computers on a network, only able to communicate with messages (so messaging came at the very beginning – it took a while to see how to do messaging in a programming language efficiently enough to be useful).
> 
> Alan Kay - twórca paradygmatu obiektowego

## Klasy w Pythonie

In [6]:
import yfinance as yf

class Stock:
    def __init__(self, ticker, name):
        self.ticker = ticker
        self._company_name = name

    @property
    def current_price(self):
        yf_stock = yf.Ticker(self.ticker)
        return self._get_current_price() or self._get_regular_market_price()

    def get_company_name(self):
        return self._company_name

    @staticmethod
    def from_ticker2(ticker):
        name = yf.Ticker(ticker).info.get("shortName")
        return Stock(ticker, name)
    
    @classmethod
    def from_ticker(cls, ticker):
        name = yf.Ticker(ticker).info.get("shortName")
        return cls(ticker, name)

    def _get_regular_market_price(self):
        return yf.Ticker(self.ticker).info["regularMarketPrice"]

    def _get_current_price(self):
        return yf.Ticker(self.ticker).info["currentPrice"]



nvda = Stock.from_ticker("NVDA")
nvda.from_ticker("MSFT")
print(f"Current price of {nvda.get_company_name()} is ${nvda.current_price:.2f}")
msft = Stock("MSFT", "Microsoft Corporation")
msft._get_current_price()

Current price of NVIDIA Corporation is $421.96


330.11

Powyżej widać przykład prostej klasy reprezentującej akcje przedsiębiorstw. Co ciekawsze jej fragmenty:
- `__init__` jest de facto tradycyjnie pojmowanym konstruktorem klasy, który działa już po zaalokowaniu pamięci na obiekt i służy do jego inicjalizacji
- wszystkie metody biorą specjalny parametr `self` reprezentujący obiekt, na którym wołana jest metoda
- `ticker` to pole publiczne
- `get_company_name` jest najzwyczajniejszą metodą publiczną (tj. będącą częścią kontraktu klasy) i zwraca wartość pola prywatnego
- `current_price` jest oznaczone jako property - w 4 od dołu linijce widać, że możemy używać tego jako pola, mimo, że jego wartość jest dynamicznie wyliczana
- `from_ticker` to metoda klasowa (w tym przypadku tzw. `factory method`), która na pierwszym parametrze bierze obiekt, reprezentujący klasę, na której została wywołana
- `_get_current_price` i `_get_regular_market_price` to metody prywatne (choć jak ktoś chce, to może je wywoływać poza tą klasą, bo Python tego nie zabrania - patrz ostatnia linijka - choć konwencja juz tak). Co do zasady ta konwencja pozwala zdefiniować stabilne publiczne API i API prywatne (prefiksowane `_`), którego stabilności nie gwarantuje się

## Podstawy dziedziczenia
Dziedziczenie klas to mechanizm pozwalający na realizację zależności, w której tzw. podklasa **jest** szczególnym przypadkiem klasy bazowej - np. akcja **jest** papierem wartościowym, ale np. akcja **ma** ticker. To ważne by odróżniać kiedy coś **jest** czymś, a kiedy coś **posiada** coś innego. Dziedziczenie stosuje się tylko gdy coś **jest** czymś, a nie po to by np. zaoszczędzic sobie pisania. Warto zachowywać zasady SOLID:
- single responsibility (tylko jeden powód do zmian)
- open/closed principle (otwartość na rozszerzanie - np. przez dziedziczenie i przedefiniowanie - oraz brak konieczności modyfikacji oryginalnej klasy)
- Liskov substitution principle - każdy egzemplarz podklasy musi pasować w każde miejsce oczekujące na obieekt klasy bazowej
- Interface segregation principle - kod zależący od danej klasy powinien zależeć tylko od potrzebnego mu interfejsu i nie powinien musieć zależeć od interfejsów mu niepotrzebnych
- Dependency inversion principle - klasy powinne zależeć od abstrakcyjnych interfejsów lub klas bazowych, a nie ich konkretnych implementacji

In [ ]:
class DataSource:
    def fetch(self, ticker, date_range):
        df = self.load_to_dataframe(ticker)
        df = self.standardize_dataframe(df)
        return self.trim_to_data_range(date_range, df)

    def load_to_dataframe(self, ticker):
        raise NotImplemented()

    def standardize_dataframe(self, df):
        df.dropna(inplace=True)
        df.sort_index(inplace=True)
        self._drop_unwanted_cols(df)
        return df

    def _drop_unwanted_cols(self, df):
        desired_cols = {"Open", "High", "Low", "Close", "Volume"}
        for col in df.columns:
            if col not in desired_cols:
                df.drop(col, axis=1, inplace=True)

    def trim_to_data_range(self, date_range, df):
        return df[(df.index >= date_range.start) & (df.index <= date_range.end)]


class LocalCSVDataSource(DataSource):
    def __init__(self: Self, data_folder: Path) -> None:
        self._data_folder = data_folder

    def load_to_dataframe(self, ticker) -> pd.DataFrame:
        try:
            file_name = f"{ticker.lower()}.us.txt"
            return pd.read_csv(
                self._data_folder / file_name, parse_dates=True, index_col=0
            )
        except FileNotFoundError as e:
            raise TickerNotFoundError() from e

## Wielodziedziczenie
Python w odróżnieniu od wielu innych nowoczesnych języków programowania wspiera wielodziedziczenie. Czym jest wielodziedziczenie? Mówiliśmy, że dziedziczenie modeluje relację `is-a`. W rzeczywistości zdarza się, że taka relacja zachodzi między różnymi bytami w zależności od poziomu abstrakcji na którym ją rozpatrujemy. Np.: samochód i rower są pojazdami, ale zarówno samochód jak i radiobudzik są urządzeniami mogącymi odtwarzać radio. W obu przypadkach relacja `is-a` zachodzi i w Pythonie możemy to explicite modelować:

In [ ]:
from typing import Self

class TradableSecurity:
    def buy(amount: int) -> int:
        ...

class EquityUnit:
    is_voting: bool
    share_ownership: int

class Stock(EquityUnit, TradableSecurity):
    def __init__(self: Self, ticker: str, price: int):
        self.ticker = ticker
        super(Stock, self).__init__(true, 1)
        

stock = Stock()
print(f"stock {'is' if stock.is_voting else 'is not'} voting")
stock.buy(10)
print(f"is Stock a TradableSecurity? {'yes' if isinstance(stock, TradableSecurity) else 'no'}")
print(f"is Stock a EquityUnit? {'yes' if isinstance(stock, EquityUnit) else 'no'}")

### super
`super` jest wbudowaną funkcją, z której możemy skorzystać by uzyskać dostęp do pól lub metod w wersjach zdefiniowanych w nadklasach - zwracany jest obiekt tymczasowy klasy bazowej (tak naprawdę tylko proxy), który może zostać "związany" z pożądaną metodą podczas jej wywołania. W swojej implementacji `super` korzysta z `mro` aby znaleźć właściwe wiązanie dla nazwy:

In [8]:
class A:
    def do_something(self):
        print("I'm A")

class B(A):
    def do_something(self):
        print("I'm B")

class C(A):
    def do_something(self):
        print("I'm C")

class D(B, C):
    def do_something(self):
        print("I'm D")

    def do_something_else_D(self):
        super(D, self).do_something()

    def do_something_else_B(self):
        super(B, self).do_something()

    def do_something_else_C(self):
        super(C, self).do_something()
        
    def do_something_else_A(self):
        super(A, self).do_something()

d = D()
d.do_something()
d.do_something_else_D()
d.do_something_else_B()
d.do_something_else_C()
d.do_something_else_A()

I'm D
I'm B
I'm C
I'm A


AttributeError: 'super' object has no attribute 'do_something'

`super()` bywa często używane gdy zachowanie override'owanej metody jest rozszerzeniem bardziej generalnego zachowania, zaimplementowanego już w klasie bazowej - szczególnie popularne jest wywoływanie konstruktora nadklasy celem inicjalizacji dziedziczonych pól.

In [ ]:
class AlphaVantageDataSource(DataSource):
    def __init__(self, api_key):
        self.__API_KEY = api_key

    def standardize_dataframe(self, df):
        df.columns = self._rename_columns(df)
        df.index = self._convert_index(df)
        super().standardize_dataframe(df)
        self._convert_columns(df)
        return df
    ...

In [ ]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)


s = Square(5)
print(f"area: {s.area()}, perimeter: {s.perimeter()}")

Skomplikowane hierarchie (wielo-)dziedziczenia rzadko są dobrym rozwiązaniem problemów i w porządnie zaprojektowanych programach spotyka się je relatywnie rzadko lub wcale. Problemy takie jak nieoczywsta kolejność wyszukiwania nazw, ale też niepotrzebne wiązanie ze sobą niepowiązanych konceptów (to, że coś jest prawdą w rzeczywistości nie oznacza, że jest dobrym modelem danej domeny w kodzie) sprawiły, że od paru lat raczej unika się modelowania kodu z użyciem wielodziedziczenia. Niektóre nowoczesne języki programowania jak Go w ogóle zrezygnowały z dziedziczenia (nie tylko wielodziedziczenia), zachęcając programistów do poszukiwania innych sposobów strukturyzowania ich problemów. Również w Pythonie pojawiło się kilka innych rozwiązań, które pozwalają na tworzenie kodu łatwiejszego w utrzymaniu i dalszym rozwoju.

## Mixins - sensowne zastosowanie wielodziedziczenia

To oczywiście nie oznacza, że wielodziedziczenie odeszło w zapomnienie - wiele frameworków używa wielodziedziczenia w sposób zgoła odmienny od modelowania relacji is-a. Zamiast tego klasy użytkownika zazwyczaj dziedziczą po klasie bazowej z frameworka oferującej podstawową funkcjonalność danego konceptu, modelując rzeczywiście tę relację, ale pozostałe klasy bazowe służą zupelnie czemu innemu. Mowa o tzw. mixins - klasach bazowych, które enkapsulują pewne zachowanie, które ciężko jednoznacznie przypisać do danego typu. Przykładowo w popularnym frameworku webowym Django możemy stworzyć widok (jako klasę) i dodać w łatwy sposób weryfikacje czy użytkownik próbujacy go użyć jest zalogowany:

In [ ]:
class MyView(LoginRequiredMixin, View):
    login_url = '/login/'
    redirect_field_name = 'redirect_to'
    ... # normal View implementation

Podobnych użyć mixinów jest wiele - mogą np. dodawać do widoku walidacje formularzy, paginację list obiektów etc. Zdarza sie również, że mixiny są wykorzystywane również poza frameworkami - właśnie wówczas gdy chcemy zenkapsulować pewne zachowania, które nie są zależne od konkretnego typu, a co najwyżej od pewnych jego atrybutów, a które duplikują się między różnymi, niepowiązanymi klasami. Czasami są to po prostu zachowania, które możemy też opakować w dekorator, jednak z racji na konieczność przechowywania stanu lub wyższy poziom komplikacji czytelniej je zenkapsulować w formie klas - np. [logowanie](https://github.com/senko/python-logger/blob/master/logger.py)

## Metody specjalne
Metody zaczynające i kończące swoje nazwy od `__` to tzw. metody specjalne. W Pythonie istnieje wiele specjalnych metod, realizujacych określone funkcje (np. umożliwiające iterowanie po klasach, porównywanie etc.) jednak na tym szkoleniu skupimy się w szczególności na metodach `__init__`, `__str__`, `__repr__`, `__eq__` i `__hash__` - czyli tych najczęściej spotykanych w kodzie aplikacji.

In [ ]:
class Stock:
    def __init__(self, ticker, name):
        self.ticker = ticker
        self._company_name = name

    def __str__(self):
        return f"{self._company_name} stock"

    def __repr__(self):
        return f"Stock[ticker: {self.ticker}, company_name: {self._company_name}]"

    def __eq__(self, other):
        return other.ticker == self.tickere and other._company_name == self._company_name

    def __hash__(self):
        return hash((self.ticker, self._company_name))


### `__str__`
Ta metoda kontroluje w jaki sposób obiekty danej klasy są wypisywanie na ekran. Celem tej metody jest przedstawienie obiektu w sposób umożliwiający *ludziom* łatwe go odczytanie, inaczej mówiąc wygoda. 

### `__repr__`
Ta metoda kontroluje w jaki sposób obiekty danej klasy są wypisywanie na ekran, jednak jej celem nie jest czytelność dla `ludzi` a jednoznaczność i potencjalnie przetwarzanie automatyczne. Wykorzystuje się ją np. w debuggerach by nie mieć wątpliwości o który obiekt chodzi - `__str__` może być niejednoznaczny o ile poprawia to czytelność np.: `str("3") == str(3)`.

### `__eq__`
`__eq__` jest wywoływana przy porównaniu z użyciem `==` - dla obiektów domyślnie rozróżnienie następuje po referencji lub hashu, jeśli obiekt jest hashowalny. Przykładowo, mamy do zaimplementowania klasę `Money` - nie ma znaczenia, która konkretnie instancja jest w naszym posiadaniu - zamiast tego liczy się np. nominał i ilość egzemplarzy, które mamy. Aby to osiągnać nadpisujemy `__eq__` tak, by zwracała `True` zawsze gdy zgadza się nominał i ilość banknotów. 

### `__hash__`
Obiekty mogą być hashowalne lub nie - jeśli mają być, to metoda specjalna `__hash__` musi być sensownie zaimplemntowana. Po co przydaje się by obiekt był hashowalny? Np. po to by mógł być kluczem w słowniku lub elementem zbioru. Aby ułatwić nam implementacje istnieje wbudowana funkcja `hash`, której można podać dowolny hashowalny argument - np. `Tuple` zawierającą pola kontrybuujące do tego, że dwa egzemplarze klasy są rozróżnialne. Jeśli egzemplarze są równe w sensie operatora `==`, to powinny mieć ten sam hash (niekoniecznie w drugą stronę). Domyślnie hashowalne są typy proste i niezmienialne kolekcje - np. `Tuple` czy `frozenset`.

### Inne metody specjalne
Wiele wbudowanych zachowań różnorakich obiektów w Pythonie jest zaimplementowana przy użyciu metod specjalnych: przykładowo, iterowanie po kolekcjach jest możliwe dzięki temu, że implementują one metody `__iter__()` i `__next__()` - korzysta z nich nawet pętla `for`! Metodami specjalnymi są operatory (np.: porównanie przy użyciu `<` jest implementowane przez `__lt__()`, mnożenie jest operatorem `__mul__` etc.), odwołania do elementów i wiele innych mechanizmów. Nie będziemy ich wszystkich omawiać, ale warto wiedzieć jak tworzyć nasze własne klasy, które mogą zyskać na naturalności użycia i być bardziej ekspresywne.

### *Zadanie*
Załóżmy przez chwilę, że nie zależy nam na szybkości obliczeń i nie chcemy używać w projekcie Pandas. W komórce poniżej zdefiniuj klasę Stock reprezentującą akcję. Jakie metody miałaby taka klasa? Czy egzemplarze tej klasy powinny być rozróżnialne, jeśli przechowują te same dane? Jeśli tak, to co powinno być jej identyfikatorem? Które metody lub które jej pola powinny być jej publicznym interfejsem? Uwzlędnij swoje odpowiedzi w kodzie poniżej:

In [1]:
class Stock:
    ...

## Dataclass
Często klasy są jedynie kontenerami na dane, bez rozbudowanych zachowań zmieniających ich wewnętrzny stan zgodnie z pewną logiką. Obiekty tych klas powinny umożliwiać łatwe tworzenie, porównywanie, wypisywanie w czytelny sposób swojej zawartości. Począwszy od Pythona 3.7 mamy do dyspozycji bibliotekę `dataclasses`, która upraszcza tworzenie i używanie takich klas - zamiast:

In [9]:
class Card:
    def __init__(self, rank, suite):
        self.rank = rank
        self.suite = suite
    
    def __eq__(self, other):
        return isinstance(other, Card) and self.rank == other.rank and self.suite == other.suite
    
    def __repr__(self):
        return f"Card({self.rank} {self.suite})"

queen_of_hearts = Card('Q', '♡')
another_queen_of_hearts = Card('Q', '♡')
assert queen_of_hearts == another_queen_of_hearts

... możemy zrobić jedynie tyle:

In [10]:
from dataclasses import dataclass

@dataclass
class Card:
    rank: str
    suite: str


queen_of_hearts = Card('Q', '♡')
another_queen_of_hearts = Card('Q', '♡')
assert queen_of_hearts == another_queen_of_hearts

Jak widać, `dataclasses` oszczędza nam sporo pisania - wystarczy, że odpowiednie pola zostaną zadeklarowane wraz ze swoim typem (konieczne! Ale może być to `Any`), a resztę zrobi za nas biblioteka. Przydatne bywa też uczynienie naszej klasy niezmienialną (*ang. mutable*) - nic prostszego! `dataclasses` czyni to naprawdę prostym:

In [11]:
from dataclasses import dataclass

@dataclass
class Card:
    rank: str
    suite: str

queen_of_hearts = Card('Q', '♡')
queen_of_hearts.rank = 'K'


@dataclass(frozen=True)
class ImmutableCard:
    rank: str
    suite: str

immutable_queen_of_hearts = ImmutableCard('Q', '♡')
immutable_queen_of_hearts.rank = 'K'

FrozenInstanceError: cannot assign to field 'rank'

Oczywiście, pola takich klas mogą mieć wartości domyślne:

In [ ]:
from dataclasses import dataclass
@dataclass
class Card:
    rank: str = get_rank()
    suite: str = '♣'

Card()

Nie należy jednak bezpośrednio wyliczać ich wywołaniem funkcji! Jest to częsty błąd w Pythonie, polegający na użyciu mutowalnych argumentów domyślnych - logika wewnątrz `dataclasses` pod spodem wygeneruje normalną klasę, zawierającą np. konstruktor (`__init__`), który dostanie podane tu wartości jako parametry. Jeśli chcemy dynamicznie lub leniwie produkować wartości domyślne możemy to zrobić tak:

In [22]:
from dataclasses import dataclass, field
import random

RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

@dataclass
class Card:
    rank: str = field(default_factory=lambda: random.choice(RANKS))
    suite: str = '♣'

Card()

Card(rank='10', suite='♣')

`field` może zmieniać różne inne parametry pola - domyślną wartość (stałą), sposób reprezentacji, porównywania, liczenia hashu etc.

**UWAGA**

In [ ]:
# tu parametr domyślny jest wyliczany tylko raz!!!
def fun1(x, lst = []):
    lst.append(x)
    return lst


print(fun1(5))
print(fun1(6))

# fix
def fun2(x, lst = None):
    if not lst:
        lst = []
    lst.append(x)
    return lst

# print(fun2(5))
# print(fun2(6))

#### Dataclasses vs namedtuple
Historycznie podobną rolę do Dataclass pełniły tzw. nazwane krotki - *namedtuples*. Tworzy się je tak:

In [ ]:
from collections import namedtuple

# Definiowanie namedtuple
Osoba = namedtuple('Osoba', ['imie', 'nazwisko', 'wiek'])

# Tworzenie instancji
jan = Osoba(imie='Jan', nazwisko='Kowalski', wiek=30)

print(jan.imie)  # Wydrukuje: Jan
print(jan[0])    # Również wydrukuje: Jan

Zalety namedtuples:
-  niezmienne (immutable), co oznacza, że ich wartości nie mogą być zmieniane po utworzeniu.
- Są mniej kosztowne pod względem pamięci niż regularne klasy.
- Dają możliwość dostępu do pól zarówno przez nazwę, jak i indeks.

Dataclasses są jednak zdecydowanie bardziej elastyczne, toteż większość projektów opartych na Pythonie 3.7 i nowszym używa Dataclass zamiast namedtuples. 

#### Dataclassess vs attrs
Attrs to biblioteka zewnętrzna, która również pozwala na definiowanie klas z automatycznym generowaniem specjalnych metod, podobnie jak dataclasses, ale jest dostępna również dla starszych wersji Pythona i oferuje nieco szerszą funkcjonalność, zwłaszcza w zakresie walidacji danych. Standardowa biblioteka dataclasses była tak naprawdę zaimplementowana na wzór attr i jest względem niej intencjonalnie okrojona. Jest natomiast dostępna w każdej instalacji (C)Pythona i używa nieco krótszej składni.

## Metaklasy

### Po co są metaklasy?
Klasy w Pythonie to obiekty i mogą być tworzone dynamicznie - wystarczy podać 3 argumenty do wbudowanej funkcji `type()` - potrzebne wówczas są kolejno: nazwa nowo tworzonego typu, tupla reprezentująca kolejne nadklasy i słownik definiujący mapowanie między identyfikatorami zawartości klasy a ich wartościami:
```
NowaKlasa = type(
...     'NowaKlasa',
...     (),
...     {
...         'atrybut1': 'coś',
...         'jakas_metoda': f
...     }
... )
```

W standardowych przypadkach, przedefiniowując metodę specjalną `__new__()` lub `__init__()`  zmieniamy sposób w jaki tworzone i inicjalizowane są obiekty. Ponieważ jednak typem obiektów reprezentujących klasy jest `type`, nadpisywanie jego metod specjalnych miałoby zdecydowanie zbyt duży efekt na już istniejące klasy. Z tego powodu próba przedefiniowania tych metod specjalnych celem zmienienia zachowania procesu tworzenia nowych typów jest zabroniona i skutkuje błędem:
```
TypeError: can't set attributes of built-in/extension type 'type'
```
Jeśli więc potrzebujemy stworzyć własną hierarchię typów, z których każdy ma takie zachowanie przedefiniowane musimy użyć metaklas, czyli stworzyć klasy dziedziczące po typie bazowym `type`. Wówczas nie ma problemu z nadpisywaniem metod specjalnych i możemy dopasować proces tworzenia klas będących obiektami takiej metaklasy bez przeszkód. Aby klasa była obiektem takiego nowego typu, musimy w jej definicji podać go w następujący sposób:
```
class MyMetaclass(type):
	def __new__(cls, name, bases, dct):
		instance = super().__new__(cls, name, bases, dct)
		instance.foo = lambda x: x ** x
		return instance

class MyClass(metaclass=MyMetaclass):
	...

MyClass().foo(2)		
```
### Zastosowania

#### ORM
Frameworki ORM często umożiwiają stworzenie klas reprezentujących modele danych, odwzorowywane później jako tabele bazy danych. W tym celu muszą umieć generować klasy reprezentujące dane jako obiekty w Pythonie, których stan jest śledzony celem automatycznego zarządzania transakcjami do bazy danych, np.: `DeclarativeMeta` w `SQLAlchemy`, która m.in/ przechwytuje przypisania wartości pól modeli i zamienia je na zapisy w bazie danych oraz śledzi z którego połączenia z bazą danych dany obiekt pochodzi.

#### FrameWorki webowe
Django, popularny framework webowy używa metaklas w wielu miejscach, przykładowo w swoich modułach ORM oraz do przetwarzania pól formularzy (`DeclarativeFieldsMetaclass`)

#### ABC
W bibliotece standardowej znajduje się moduł `abc`, który udostępnia klasę bazową ABC. Klasy po niej dziedziczące są wówczas traktowane jako abstrakcyjne klasy bazowe i nie możliwe jest utworzenie ich instancji - zamiast tego dopiero ich podklasy, implementujące określone metody abstrakcyjne mogą być tworzone

Oprócz tego metaklasy znajdują zastosowanie w przypadkach, gdy sam process tworzenia klas wymaga przedefiniowania - np. by śledzić ich obiekty, walidować porządane zachowanie dużych hierarchii typów etc.

In [ ]:
class Cokolwiek:
    def __init__(self):
        self.a = 5
        self.b = "blabla"
        
    def powtorz(self):
        return self.b * self.a

class Cokolwiek2:
    def __init__(self):
        self.c = []

    def dodaj_do_c(self, x)
        self.c = x


Należy jednak pamiętać, że zazwyczaj istnieją prostsze sposoby osiągnięcia podobnych efektów. Metaklasy są bardzo zaawansowaną funkcjonalnością języka i zazwyczaj nie powinne być używane przez 99% użytkowników. Jeśli ktoś waha się czy użyć metaklas to prawdopodobnie nie ma use-case, który by ich wymagał.


### *Zadanie*

## UWAGA ZATEGOWANIE
```
// Skopiuj gdzieś na zewnątrz test_tickers_*.txt
git checkout task-metaclasses
git checkout -b metaclasses-task-solution
```
Podklasy `DataSource` w pakiecie `stock_trader.acquisition.data_sources` w module `data_source` mają metodę `fetch`, która jest ich główną funkcjonalnością - zwraca `DataFrame` z `pandas` zawierający kolumny `OHLCV`. `OHLCV` to inaczej `Open High Low Close`, czyli ceny akcji z początku dnia, najwyższą i najniższą dzienną cenę i cenę z końca dnia. `V` oznacza wolumen akcji. Kod aplikacji w kilku miejscach zależy od tego, że `DataFrame` wczytany przez `DataSource` zawiera te kolumny, jednak naszym celem jest uniknięcie sprawdzania tego w każdym miejscu jego użycia. 
- Użyj metaklas do zwalidowania, że odpowiednie kolumny istnieją w zwracanym z `fetch` obiekcie
- Przetestuj stworzone rozwiązanie
- Jaki inny mechanizm może pomóc osiągnąć podobny cel? Zaimplementuj z jego użyciem analogiczne rozwiązanie
- Odpal wszystkie testy stock_tradera - czy pojawiają się jakieś nietypowe błędy? Jak możemy ich uniknąć, pozostając przy rozwiązaniu opartym o metaklasy?

## Modularyzacja projektów
Gdy projekty rozrastają się, pojawia się potrzeba podzielenia ich na pakiety i zaprowadzenia w nich pewnego porządku, który:
- umożliwia łatwą nawigację po bazie kodu
- umożliwia możliwie jasne określenie, które elementy są interfejsem publicznym nie tylko klas, ale również pakietów
- zarządzanie zależnościami między pakietami tak, by zmniejszać ich *coupling* i umożliwiać w miarę bezproblemową kolaborację wielu zespołów nad wspólnym repozytorium
- elastyczną wymianę implementacji abstrakcji, bez konieczności zmiany po stronie modułów, które od nich zależą

Nie istnieje jeden sposób na osiągnięcie tych celów dla dowolnej aplikacji. W zależności od domeny i rodzaju problemu można wykorzystać architekturę warstwową jak MVC, architekturę hexagonalną czy podejście pipes and filters. Przykładowa hierarchia dla aplikacji w architekturze hexagonalnej wygląda tak:
```
projekt
|--src
|     |-- bounded_context1
|     |                  |-- adapters
|     |                  |           |-- orm.py
|     |                  |           |-- orm_test.py # integration tests
|     |                  |           |-- repository.py 
|     |                  |           |-- repository_test.py # unit/integration test
|     |                  |-- domain
|     |                  |         |-- model.py 
|     |                  |         |-- model_test.py # unit tests
|     |                  |-- entrypoints
|     |                  |              |-- flask_app.py
|     |                  |              |-- flask_app_test.py # e2e tests
|     |                  |-- service_layer
|     |                  |                |-- service.py 
|     |                  |                |-- service_test.py # integration tests with stubbed environment
|     |                  |-- settings.py
|     |-- bounded_context2
|     |                  |-- adapters
|     |                  |           |-- dynamodb.py
|     |                  |           |-- dynamodb.py # integration tests
|     |                  |           |-- repository.py 
|     |                  |           |-- repository_test.py # unit/integration test
|     |                  |-- domain
|     |                  |         |-- model.py 
|     |                  |         |-- model_test.py # unit tests
|     |                  |-- entrypoints
|     |                  |              |-- sqs.py
|     |                  |              |-- sqs_test.py # e2e tests
|     |                  |-- service_layer
|     |                  |                |-- service_test.py # integration tests with stubbed environment
|     |                  |                |-- service.py 
|     |                  |-- settings.py
```
Oczywiście powyższy diagram należy raczej traktować jako zachętę do rozmowy - szczegóły należy dopasować do potrzeb projektu. Nie ma powodu by np. zakładać, że dokładnie taka sama architektura jest konieczna w każdym z implementowanych przez daną usługę bounded contextów - być może bounded_context2 jest niewielkim CRUDem. Warto zadbać, by bounded contexty w tak zorganizowanym kodzie nie importowały kodu między sobą, a co najwyżej "rozmawiały" ze sobą wołając się nie niżej niż przez warstwę usług. 

Ten sposób organizacji projektu ma również inną zaletą - narzędzia takie jak setuptools umieją automatycznie odnajdywać jego zawartość, zmniejszając czas potrzebny na ich konfigurację.

Dla projektu implementującego potok przetwarzania, składający się z kolejnych etapów możemy posłużyć się architekturą, gdzie kolejne etapy przetwarzania są osobnymi pakietami, a zależności między modułami są tylko poziomie abstrakcji umożliwiających wejście lub wyjście z etapu.
Ważne jest jednak tworzenie "bąbli czystego kodu" - rzadko który projekt w długiej perspektywie utrzymuje swój oryginalny kształt i na koniec prawie zawsze gdzieś pojawia się dług techinczny. Odizolowane pakiety, mające mało zależności zewnętrznych są łatwiejsze w utrzymaniu i testowaniu. Przykładem takiego pakietu może być pakiet `utils` lub `trading_algorithms`, które praktycznie nie mają zewnętrznych zależności w projekcie. Ich rozwijanie jest proste, a zmiany w nich - pod warunkiem zachowania interfejsów ich klas bazowych nie powodują zmian w reszcie kodu.

## Spoistość (*cohesion*)

Spoistość odnosi się do stopnia, w którym elementy wewnątrz modułu lub klasy są powiązane ze sobą. Wysoki wskaźnik spoistości oznacza, że wszystkie funkcje i zmienne w module lub klasie są ściśle powiązane i współpracują ze sobą w realizacji jednego zadania. 
Jak osiągnąć wysoką spoistość?
- Upewnij się, że każda klasa lub moduł wykonuje tylko jedno zadanie.
- Unikaj klas "wszystko-w-jednym", które mają zbyt wiele odpowiedzialności.
- Podziel duże klasy lub moduły na mniejsze, bardziej skoncentrowane jednostki.

### Ocenianie spoistości *na oko*
Jedną z prostszych heurestyk przy ocenie spoistości klas jest zwrócenie uwagi na to jaka część metod klasy używa jakiej części jej pól z danymi. Idealnie jeśli każde pole jest używane w każdej metodzie. Jeśli natomiast pola można pogrupować tak, że pewien podzbiór metod korzysta tylko z danej grupy pól to jest to mocne wskazanie do refaktoryzacji i wydzielenia nowej klasy. Można następnie użyć kompozycji do połączenia obu klas. Warto zwrócić uwagę na to, czy obiekty wydzielanej klasy mają jakąś niezależną od danych tożsamość. Jeśli nie, warto je uczynić niezmienialnymi (*immutable*) - w terminologii DDD mamy wówczas do czynienia z tzw. *Value Objectem*. W połączeniu z zasadą by każdy taki obiekt był zawsze poprawny (niepoprawne obiekty nie powinny móc być stworzone) dostajemy wówczas mechanizm znacząco poprawiający czytelność i utrzymywalność kodu. Jest to również technika, którą warto jako pierwszą zaprząc gdy mamy do czynienia z kodem, który uważamy za posiadający duży dług techniczny. Zwykle po paru iteracjach takiego wyłączania jest nam dużo łatwiej nawigować bazę kodu i dokonywać bardziej zaawansowanych refaktoryzacji.

Analogicznie dla pakietów należy przeanalizować, które klasy powinny być eksportowane i czy pozostałe klasy w pakiecie służą wyłącznie do ich implementacji. Jeśli jest wiele klas i prawie każda jest przeznaczona do publicznego użycia, być może niektóre z nich zasługują na bycie w osobnym pakiecie. W ostateczności możemy też kontrolować klasy od których pozwalamy reszcie kodu zależeć przy użyciu zmiennej specjalnej `__all__` w pliku `__init__.py` w danym pakiecie. 

### Bardziej naukowe podejście do spoistości
Powyższa heurystyka to uproszczenie metryki LCOM. LCOM to skrót od "Lack of Cohesion in Methods" (Brak Spoistości w Metodach). Jest to metryka używana w inżynierii oprogramowania do pomiaru spoistości klasy. Spoistość odnosi się do stopnia, w jakim składniki w obrębie pewnej jednostki (takiej jak klasa) są powiązane ze sobą. Klasy o wysokiej spoistości mają metody, które są ściśle powiązane z jej stanem, podczas gdy klasy o niskiej spoistości mają metody, które nie są mocno powiązane z jej stanem.

LCOM jest używane do oceny, jak dobrze metody w klasie są powiązane z polami tej klasy. Istnieje kilka wariantów tej metryki (np. LCOM1, LCOM2, LCOM3, LCOM4), ale koncepcja jest podobna. Niestety w ekosystemie Pythona nie jest to szczególnie popularna praktyka - istnieje dość [amatorski projekt na PyPI](https://pypi.org/project/lcom/), natomiast pewnie lepsze wyniki przyniesie napisanie własnej wtyczki do Sonara. Jest to jednak dość żmudna praca i najczęściej rzadko uzasadniona względem heurystyki *na oko*.

### Zalety dbania o spoistość
Oto kilka powodów, dla których warto dbać o wysoką spoistość:

- **Zrozumiałość**: Moduły lub klasy o wysokiej spoistości są zazwyczaj bardziej zrozumiałe, ponieważ koncentrują się na jednym zadaniu lub funkcji. Dzięki temu programiści mogą szybciej zrozumieć, co dany fragment kodu robi, nie przekopując się przez wiele niepowiązanych funkcji czy zmiennych.
- **Łatwiejsze Utrzymanie**: Gdy moduł lub klasa ma tylko jedną odpowiedzialność, wprowadzenie zmian staje się prostsze. Jeśli chcesz coś zmienić, zmieniasz tylko ten konkretny moduł, a nie wiele innych miejsc w kodzie.
- **Reużywalność**: Moduły o wysokiej spoistości są zazwyczaj łatwiejsze do ponownego użycia w innych projektach lub częściach tego samego projektu, ponieważ mają jedno, dobrze zdefiniowane zadanie.
- **Mniejsze Ryzyko Błędów**: Kiedy kod ma wysoką spoistość, jest mniejsza szansa, że wprowadzenie zmian w jednej części systemu niechcący wpłynie na inną część systemu. Możesz unikać nieprzewidzianych skutków ubocznych.
- **Lepsza Modularność**: Spoistość sprzyja tworzeniu kodu modularnego. Dzięki modularności, system można łatwiej rozbudowywać, dodając nowe moduły zamiast modyfikować istniejące.
- **Łatwiejsze Testowanie**: Moduły lub klasy o wysokiej spoistości są łatwiejsze do testowania, ponieważ koncentrują się na jednym zadaniu. Można łatwo izolować je i pisać testy jednostkowe, które skupiają się na konkretnych funkcjonalnościach.

## Pytest i zarządzanie zależnościami - demo
- fixtures
    - scopes
- built-in fixtures: monkeypatch, tmp_path
- odpalanie wybranych testów
- 

## Wzorce projektowe

### Singleton - antywzorzec
Czasami chcemy mieć dokładnie jedną kopię danego obiektu w całej aplikacji. Wówczas z pomocą przychodzi wzorzec "singleton", który zapewnia, że podczas działania programu obiekt danej klasy zostanie utworzony dokładnie raz. Można go zaimplementować w następujący sposób:

In [2]:
class Singleton:
    _instance = None

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super(Singleton, cls).__new__(cls, *args, **kwargs)
        return cls._instance

# Usage:

singleton1 = Singleton()
singleton2 = Singleton()

print(singleton1 == singleton2)  # True, since they are the same instance


True


Najczęściej jednak użycie takiego wzorca wiąże się ze złą organizacją kodu i powoduje liczne problemy:
- **Globalny stan**: Singletony często reprezentują globalny stan w aplikacji. Może on wprowadzać ukryte zależności między klasami, co sprawia, że system jest trudniejszy do zrozumienia i utrzymania.
- **Trudniejsze testowanie**: singletony mogą utrudniać testowanie jednostkowe, ponieważ trzymają w sobie stan przez cały czas życia aplikacji. Zachowany stan może prowadzić do efektów ubocznych, które czynią testy nieprzewidywalnymi. Co więcej, trudno jest zastąpić Singletona mockiem lub stubem (zazwyczaj konieczny jest monkeypatching), co prowadzi do problemów z testowaniem komponentów niezależnie od siebie.
- **Problemy z wielowątkowością**: W środowiskach wielowątkowych zapewnienie, że instancja Singletona jest tworzona tylko raz, może być trudne lub wręcz niemożliwe. Bez odpowiedniej synchronizacji istnieje ryzyko stworzenia wielu instancji, co jest sprzeczne z celem wzorca i ma niezerowy wpływ na wydajność
- **Skalowalność**: W systemach rozproszonych koncepcja "jednej instancji" staje się wyzwaniem. Jeśli aplikacja korzystająca z Singletona musi być skalowana na wielu serwerach, bardzo trudne jest zapewnie, że istnieje tylko jedna instancja Singletona, unikalna w całym systemie.
- **Użycie pamięci**: Ponieważ instancja Singletona często jest tworzona i zachowywana przez cały czas życia aplikacji (natychmiastowa inicjalizacja), może prowadzić do niepotrzebnego zużycia pamięci, jeśli instancja jest ciężka i nie jest wymagana przez cały czas działania aplikacji.

Nie oznacza to jednak, że nigdy nie wolno z takiego rozwiązania korzystać - przykładowo ciągnięcie zależności od globalnych ustawień przez kilka warst aplikacji może być mniej elegancką opcją niż zaimportowanie singletona z modułu. Warto wtedy jednak posłużyć się innym mechanizmem i skorzystać z własności systemu importów, gwarantującego jednokrotną ich inicjalizację (patrz `APP_SETTINGS` w module `settings.py`)
### Dependency injection and inversion a testowalność klas
Aby ułatwić testowanie i uniknąć konieczności mockowania zależności klas, możemy posłużyć się wzorcem wstrzykiwania zależności *dependency injection*. W skrócie, polega to na unikaniu tworzenia instancji obiektów wewnątrz konstruktora czy też metod klasy, a zamiast tego przyjmowaniu ich jako parametrów konstruktora i zapisywania referencji do nich w stanie obiektu. Dzięki temu podczas testowania łatwo możemy podać fake'ową implementację, która wypełnia kontrakt od którego zależy testowana klasa, a która unika efektów ubocznych i wykonuje się szybciej, lepiej izolując nasz test.
*Przykład: data_loader_test.py*

Ta technika działa wyjątkowo dobrze w połączeniu z zasadą odwrócenia zależności, której formalna definicja brzmi:
1. Moduły wysokopoziomowe nie powinny zależeć od modułów niskopoziomowych. Oba typy powinny zależeć wyłącznie od abstrakcji
2. Abstrakcje nie powinny zależeć od szczegółowych implementacji. Przeciwnie: szczegółowe implementacje powinny zależeć od abstrakcji

Czym są owe moduły wysokopoziomowe? W skrócie: kodem domenowym, warstwą implementującą logikę biznesową. Stosując odwrócenie i wstrzykiwanie zależności można osiągnąć wysoki stopień pokrycia kodu testami (zwiększając zaufanie do ważnego dla organizacji kodu) bez spowolnienia ich wydajności (bo testy są jedynie jednostkowe).

Moduły niskopoziomowe to z kolei kod stricte techniczny - adaptery kolejek w systemach rozproszonych, połączenia do bazy czy kod czytający pliki. Ich zależność od modułów wysokopoziomowych powinna istnieć wyłącznie na poziomie importowania definiowanych tam interfejsów - abstrakcyjnych klas bazowych.

Sens 2 punkty definicji jest natomiast taki: izolując kod domenowy od detali implementacji sposobu przechowywania danych lub komunikowania się ze światem zewnętrznym, pozwalamy im się zmieniać w odpowiedzi na często niezależne czynniki. Powody do zmiany kodu biznesowego są najczęściej inne niż powody do zmiany kodu technicznego i odbywają się w innej kadencji.

### Metoda fabryczna
https://refactoring.guru/design-patterns/factory-method/python/example

Dodatkowo, w Pythonie nie ma przeciążania funkcji i metod, więc czasem konieczne jest stworzenie metod fabrycznych by zaimplementować więcej sposobów tworzenia klas
### Fabryka i fabryka abstrakcyjna
Uproszczenie inicjalizacji obiektów - https://refactoring.guru/design-patterns/abstract-factory/python/example + visualizer_factory w kodzie stock tradera

### *Zadanie*
```
git checkout design-patterns-task
git checkout -b design-patterns-task-solution
```
Visualizer to klasa, której odpowiedzialnością jest rysowanie wykresów określonych wskaźników technicznych, które zostały obliczone przy użyciu klas z pakietu `trading_algorithms`. Jej interfejs, od którego zależy m.in. `PlottingWorkflow` to:
```
class Visualizer:

    def plot(self: Self, data: pd.DataFrame, ticker: str, indicator_names: list[str]) -> None:
        ...
  
```

Zaimplementuj klasę (w szczególności metodę `plot`) tak, by móc rysować przynajmniej 2 wskaźniki:
- [ ] MovingAverageConvergenceDivergence
- [ ] RelativeStrengthIndicator

Załóż, że `DataFrame` data ma je już wyliczone w kolumnach, których nazwy są przekazane w liście `indicator_names`. Pamiętaj, że jeden typ wskaźnika może wymagać >=1 linii lub słupków. Załóż, że dane są poprawne. Aby narysować wykres, należy użyć `matplotlib` - najlepiej przez wrapper `mplfinance`.
Każdy wykres należy narysować poniżej wykresu danych OHLC. Przykładowo, aby narysować wykres samego kursu można wywołać:
```
 mpf.plot(
            data[["Open", "High", "Low", "Close", "Volume"]],
            type="candle",
            style="yahoo",
            volume=True,
            title=f"stock price for {ticker}",
            savefig="output.png"
        )
```

Dodatkowe podwykresy możemy dodawać przy użyciu parametru `addplot`, którego wartości możemy konstruować tak:
```
 mpf.make_addplot(data[indicator_name], panel=1, ylabel=indicator_name)
```

Więcej przykładów można znaleźć [tu](https://github.com/matplotlib/mplfinance/tree/master/examples)

W pliku `src/stock_trader/workflows/plotting.py` można znaleźć `PlottingWorkflow`, który korzysta z obiektów `Visualizera` by narysować żądany przez użytkownika wykres. W pliku `src/stock_trader/cli/commands.py` `PlottingWorkflow` jest tworzony - tam można dopasować sposób tworzenia `Visualizera`

Dodatkowo:
- [ ] jak poradzić sobie z rosnącymi wymaganiami ze strony biznesu, żądającego implementacji rysowania wielu innych typów wskaźników? Np. Boilinger Bands, On Balance Volume etc.?
- [ ] czy klasa, którą zaimplementowaliście jest spoista? czy jest czytelna?


### Dekorator i Łańcuch odpowiedzialności
https://refactoring.guru/design-patterns/chain-of-responsibility + visualizer na 2 sposoby